In [36]:
#Libraries Needed For NLTK
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

#Libraries For Tenserflow Processing
import tensorflow as tf 
import numpy as np 
import random 
import json

[nltk_data] Downloading package punkt to C:\Users\Harsh
[nltk_data]     Patel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
# Import our Chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [38]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [39]:
words = []
classes = []
documents = []
ignore = ['?']

# Loop through each intent in the intents
for intent in intents['intents']:
    # Loop through each sentence pattern in the current intent
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence pattern
        w = nltk.word_tokenize(pattern)
        
        # Add words to the word list
        words.extend(w)
        
        # Add words and their corresponding intent tag to the documents
        documents.append((w, intent['tag']))
        
        # Add the intent tag to the classes list if not already present
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [40]:
# Perform stemming and lower each word as well as remove duplicates

from enum import unique


words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(classes)))

#remove duplicate classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes) ,'classes' , classes)
print(len(words) , 'unique stemmed words',words)

27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
8 unique stemmed words ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']


In [41]:
#Create training Data
training =[]
output = []

#create an empty array for output
output_empty = [0]*len(classes)

#create training set,bag of words for each sentence
for doc in documents:
    #initialize bag of words
    bag=[]
    #list of tokenized words for the pattern
    pattern_words = doc[0]
    # streaming each words
    pattern_words= [stemmer.stem(word.lower()) for word in pattern_words]
    #create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    #output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag,output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training list

train_x = list(training[:,0])
train_y = list(training[:,1])

In [42]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10 , input_shape = (len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation = 'softmax'))
model.compile(tf.keras.optimizers.Adam() , loss='categorical_crossentropy' , metrics=['accuracy'])

In [43]:
model.fit(np.array(train_x) , np.array(train_y) , epochs=100 , batch_size = 20 ,verbose=1)
model.save("model.api")

Epoch 1/100
2/2 [==============================] - 2s 14ms/step - loss: 2.0928 - accuracy: 0.0741
Epoch 2/100
2/2 [==============================] - 0s 12ms/step - loss: 2.0883 - accuracy: 0.1852
Epoch 3/100
2/2 [==============================] - 0s 11ms/step - loss: 2.0837 - accuracy: 0.1852
Epoch 4/100
2/2 [==============================] - 0s 16ms/step - loss: 2.0804 - accuracy: 0.1852
Epoch 5/100
2/2 [==============================] - 0s 19ms/step - loss: 2.0764 - accuracy: 0.1852
Epoch 6/100
2/2 [==============================] - 0s 11ms/step - loss: 2.0727 - accuracy: 0.1852
Epoch 7/100
2/2 [==============================] - 0s 22ms/step - loss: 2.0684 - accuracy: 0.2593
Epoch 8/100
2/2 [==============================] - 0s 12ms/step - loss: 2.0649 - accuracy: 0.2593
Epoch 9/100
2/2 [==============================] - 0s 16ms/step - loss: 2.0618 - accuracy: 0.2593
Epoch 10/100
2/2 [==============================] - 0s 28ms/step - loss: 2.0574 - accuracy: 0.2593
Epoch 11/100
2/2 [=

INFO:tensorflow:Assets written to: model.api\assets


In [ ]:
import pickle
pickle.dump({})